In [294]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [295]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [296]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Kamil,Biernacki,87120919896,Wrocław,ul. Rzepakowa 8,48954462012,kamil.biernacki@kuba.pl,2014-01-01,None,menadżer,73.5
1,2,Wojciech,Słomski,62040579731,Wrocław,ul. Krótka 79/13,48397916657,wojciech.slomski@kuba.pl,2014-01-01,None,sprzedawca,48.9
2,3,Jarosław,Gawłowicz,66040246170,Wrocław,ul. Campi 26,48624664626,jaroslaw.gawlowicz@kuba.pl,2014-01-01,2017-09-08,sprzedawca,45.0
4,4,Danylo,Szymczak,62112231974,Bydgoszcz,ul. Ceglana 46,48961214659,danylo.szymczak@kuba.pl,2014-01-01,None,mechanik,57.7
12,5,Andrzej,Jankowski,94092541318,Poznań,ul. Zielone Wzgórze 13,48522112726,andrzej.jankowski@kuba.pl,2014-01-01,None,informatyk,42.6
3,6,Helena,Hoffmann,88120246487,Wrocław,ul. Słoneczna 73/45,48517206292,helena.hoffmann@kuba.pl,2015-03-01,None,sprzedawca,56.2
5,7,Kazimierz,Czyżewski,75022818692,Wrocław,ul. gen. Stanisława Maczka 40/11,48192893700,kazimierz.czyzewski@kuba.pl,2016-05-08,None,mechanik,49.6
6,8,Olena,Gawrońska,83021189806,Wrocław,ul. Lipowa 14/19,48163674793,olena.gawronska@kuba.pl,2016-05-24,None,mechanik,54.8
7,9,Wanda,Małyszek,90101079263,Krosno,ul. Jana Pawła II 18,48881018769,wanda.malyszek@kuba.pl,2016-08-27,None,mechanik,46.9
8,10,Małgorzata,Skop,77062535302,Wrocław,ul. Skalna 92/41,48204989709,malgorzata.skop@kuba.pl,2017-01-15,None,mechanik,50.6


In [297]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [298]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [299]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [300]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [301]:
len(year_ceil), len(pd.unique(client_ids))

(1404, 1405)

In [302]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [303]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [304]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(764, 758, 491)

In [305]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [306]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [307]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [308]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [309]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Kamil,Biernacki,M,87120919896,Wrocław,ul. Rzepakowa 8,False,48954462012,kamil.biernacki@kuba.pl
1,1,Małgorzata,Stelmasiak,K,77061576276,Sulechów,ul. Pogodna 52/34,False,48284925275,malstelmasi1977@interia.pl
2,2,Adam,Waśko,M,88090978795,Kętrzyn,ul. Dobra 67/13,False,48849618078,wasko1@onet.pl
3,3,Bogusława,Żbik,K,67030151795,Wrocław,ul. Felczyńska 78/35,False,48759780060,kqwbzlm0ta01@opayq.com
4,4,Adrian,Zarytskyi,M,61112329474,Wejherowo,ul. Błońska 24,True,48627915491,azarytskyi1961@o2.pl
...,...,...,...,...,...,...,...,...,...,...
1400,1400,Wojciech,Gurtowski,M,86050644052,Rogoźno,ul. Kazimierza Wielkiego 12/22,False,48356312470,w-gurtowski86@wp.pl
1401,1401,Bożena,Noak,K,68100867714,Wrocław,ul. Jastrzębska 70,False,48717875739,noak8204@vp.pl
1402,1402,Miłosz,Kowalski,M,67100361259,Wrocław,ul. Chabrowa 58/10,False,48065256633,mil.kowal67@gmail.com
1403,1403,Artur,Duda,M,None,Wrocław,ul. Św. Ojca Pio 79/6,False,48983075033,aduda@onet.pl


In [310]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [311]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Kamil,Biernacki,M,87120919896,Wrocław,ul. Rzepakowa 8,False,48954462012,kamil.biernacki@kuba.pl
1,1,Małgorzata,Stelmasiak,K,77061576276,Sulechów,ul. Pogodna 52/34,False,48284925275,malstelmasi1977@interia.pl
2,2,Adam,Waśko,M,88090978795,Kętrzyn,ul. Dobra 67/13,False,48849618078,wasko1@onet.pl
3,3,Bogusława,Żbik,K,67030151795,Wrocław,ul. Felczyńska 78/35,False,48759780060,kqwbzlm0ta01@opayq.com
4,4,Adrian,Zarytskyi,M,61112329474,Wejherowo,ul. Błońska 24,True,48627915491,azarytskyi1961@o2.pl
...,...,...,...,...,...,...,...,...,...,...
1400,1400,Wojciech,Gurtowski,M,86050644052,Rogoźno,ul. Kazimierza Wielkiego 12/22,False,48356312470,w-gurtowski86@wp.pl
1401,1401,Bożena,Noak,K,68100867714,Wrocław,ul. Jastrzębska 70,False,48717875739,noak8204@vp.pl
1402,1402,Miłosz,Kowalski,M,67100361259,Wrocław,ul. Chabrowa 58/10,False,48065256633,mil.kowal67@gmail.com
1403,1403,Artur,Duda,M,None,Wrocław,ul. Św. Ojca Pio 79/6,False,48983075033,aduda@onet.pl


In [312]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [313]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(68)

In [314]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(205)

# auta

In [315]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [316]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [317]:
df_samochody = create_car_table(max(car_ids))

In [318]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Opel,Astra,4,2007,Hatchback,False,czarny,Diesel,2.2,136,Manualna,5,2,False
1,2,Dacia,Sandero,4,2007,Hatchback,False,szary,Benzyna,1.2,73,Manualna,5,5,False
2,3,Dacia,Duster,4,2012,Sedan,False,czerwony,Benzyna,1.3,90,Manualna,5,5,False
3,4,Volkswagen,Golf Cabriolet,4,2003,SUV,False,czarny,Benzyna,3.4,211,Manualna,5,2,False
4,5,Renault,ZOE,4,2007,Hatchback,False,biały,Elektryczny,1.3,88,Automatyczna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,1004,Volkswagen,Tiguan,4,2010,Sedan,False,czarny,Benzyna,2.3,150,Manualna,5,5,False
1004,1005,Skoda,Superb,4,2007,SUV,False,szary,Diesel,2.5,160,Automatyczna,5,5,True
1005,1006,Mercedes-Benz,E 400,4,2009,SUV,False,biały,Diesel,5.3,340,Automatyczna,5,5,True
1006,1007,Nissan,Micra,4,2015,Kombi,False,czarny,Benzyna,1.6,98,Manualna,5,2,False


In [319]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [320]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Manualna', 'Automatyczna', 'Półautomatyczna'], dtype=object)

# Usługi

In [321]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [322]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [323]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [324]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_14068\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [325]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [326]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,47000.0
1,2,2,2,6,2014-01-02,29800.0
2,3,3,3,3,2014-01-02,56900.0
3,4,4,4,3,2014-01-02,55700.0
4,5,5,5,3,2014-01-02,28800.0
...,...,...,...,...,...,...
759,760,1002,1400,2,2017-12-24,43500.0
760,761,1003,1401,2,2017-12-26,62800.0
761,762,1004,1390,2,2017-12-26,59300.0
762,763,1005,1402,2,2017-12-28,49000.0


In [327]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [328]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [329]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [330]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_14068\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [331]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,1,21,2,66606.0,6,2014-01-16
1,2,6,22,3,70686.0,6,2014-01-16
2,3,18,23,6,49776.0,6,2014-01-17
3,4,7,24,6,111904.0,12,2014-01-17
4,5,19,25,3,184970.0,18,2014-01-18
...,...,...,...,...,...,...,...
753,754,993,643,6,56888.0,12,2017-12-22
754,755,987,250,3,20900.0,1,2017-12-22
755,756,1000,1282,3,12300.0,1,2017-12-23
756,757,1001,1142,3,53456.0,12,2017-12-26


In [332]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [333]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [334]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(29), array([23])), (np.int64(31), array([2])), (np.int64(33), array([4])), (np.int64(34), array([62, 26,  9])), (np.int64(35), array([61, 12])), (np.int64(36), array([4, 0])), (np.int64(37), array([ 8, 16])), (np.int64(38), array([40, 20])), (np.int64(39), array([27])), (np.int64(41), array([16])), (np.int64(45), array([11])), (np.int64(52), array([33])), (np.int64(55), array([54, 43, 19])), (np.int64(56), array([57])), (np.int64(60), array([22])), (np.int64(61), array([ 2, 20,  5])), (np.int64(62), array([54, 58])), (np.int64(64), array([28])), (np.int64(69), array([15, 62])), (np.int64(71), array([50])), (np.int64(72), array([50, 23, 37])), (np.int64(75), array([48, 62])), (np.int64(77), array([57, 18])), (np.int64(79), array([ 3, 47])), (np.int64(81), array([19, 37])), (np.int64(82), array([11])), (np.int64(83), array([61])), (np.int64(84), array([57, 41])), (np.int64(88), array([ 9, 42])), (np.int64(93), array([36])), (np.int64(91), array([17])), (np.int64(95), array([20

In [335]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [336]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Podświetlenie tablicy,,20,1,1
1,Kratka,Suzuki SX4 niebieski,257,2,1
2,Felga,R15 niebieski,91,3,4
3,Lusterko zewnętrzne prawe,SEAT Mii czarny,17,4,1
4,Poduszka powietrzna,,100,4,1
...,...,...,...,...,...
741,Rozrusznik,Ford Fiesta,302,488,1
742,Rura wydechowa,,100,489,1
743,Lakier,czarny,202,490,1
744,Zawór EGR,,500,491,1


# Wyposażenie

In [337]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [338]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Podświetlenie tablicy,,20.0
1,2,Kratka,Suzuki SX4 niebieski,257.0
2,3,Felga,R15 niebieski,91.0
3,4,Lusterko zewnętrzne prawe,SEAT Mii czarny,17.0
4,5,Poduszka powietrzna,,100.0
...,...,...,...,...
734,549,Hamulec ręczny,BMW 120,69.0
736,550,Deska rozdzielcza,Opel Astra,214.0
737,551,Zderzak,Opel Astra biały,353.0
740,552,Lusterko zewnętrzne lewe,Bajaj Xcd niebieski,4.0


# Użyte części - uzupełnienie

In [339]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df['cena'] = pd.to_numeric(used_parts_df['cena'])
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość', 'cena']]

In [340]:
used_parts_df

,id,id_usługi,id_części,ilość,cena
0,1,1,1,1,20
1,2,2,2,1,257
2,3,3,3,4,91
3,4,4,4,1,17
4,5,4,5,1,100
...,...,...,...,...,...
741,742,488,553,1,302
742,743,489,52,1,100
743,744,490,61,1,202
744,745,491,38,1,500


In [341]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [342]:
# def get_service_date(part_id):
    
#     index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
#     service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
#     return service_date
    

# buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
# parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

# for i in range(len(buy_parts_quantity)):
#     part_id = buy_parts_quantity['id_części'][i]
#     quantity = buy_parts_quantity['ilość'][i]
#     price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
#     total_cost = np.floor(price * rng.uniform(0.75, 0.95))[0]
#     service_date = get_service_date(part_id)
    
#     row = [int(i+1), int(part_id), total_cost, int(quantity)]
#     parts_purchase_pd.loc[i] = row


In [343]:
# parts_purchase_pd

In [344]:
# parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [345]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [346]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi uzupełnienie

In [347]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.25, 1.7), -1).astype(int)

In [348]:
df_services.to_csv("outputs/usługi.csv", index=False)